In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata, select_images
from models import County, DataTrustedIdentifier

In [3]:
engine, session=db.connect()
engine.echo=False

2023-04-03 14:08:40,031 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-04-03 14:08:40,031 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-03 14:08:40,032 INFO sqlalchemy.engine.Engine select current_schema()
2023-04-03 14:08:40,033 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-03 14:08:40,034 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-04-03 14:08:40,034 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens/pr_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']

query = session.query(County).filter(County.uf.__eq__('PR')).distinct().all()
state_unaccented_lower, uf_unaccented_lower, county_unaccented_lower=db.get_state_uf_county(query)

In [6]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            list_level = []
            list_path_images_final = []
            list_seq_final = []
            list_count_samples = []

            level=DataTrustedIdentifier.specific_epithet_trusted
            out=os.path.join(path_out, color.upper(), level.name, image_size, str(minimum_image))


            condition = sa.and_(DataTrustedIdentifier.country_trusted.__eq__('Brasil'),
                                    level.is_not(None),
                                    sa.or_(sa.and_(uf_unaccented_lower,
                                           state_unaccented_lower,
                                           county_unaccented_lower),
                                           sa.and_(uf_unaccented_lower,
                                           county_unaccented_lower),
                                           sa.and_(state_unaccented_lower,
                                           county_unaccented_lower),
                                           uf_unaccented_lower,
                                           state_unaccented_lower)
                                    )


            query = db.get_records_group_by_level(condition, level, minimum_image, session)
            print('color: %s image_size: %s minimum_image: %d' % (color.upper(), image_size, minimum_image))
            select_images(color, image_size, list_count_samples, list_level, list_images_invalid, list_path_images_final, list_seq_final, minimum_image, session, query)
            

            list_f = copy_images(list_level, list_path_images_final, out)
            save_metadata(color, image_size, level, list_count_samples, list_f, list_level, list_path_images_final, list_seq_final, minimum_image, out, session)

color: RGB image_size: 256 minimum_image: 5
total of levels: 55 total of images: 1356


,levels,count,dst,paths,seq
0,aduncum,33,/home/xandao/Imagens/pr_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[9674, 9927, 10002, 10051, 10094, 10468, 10676..."
1,alata,18,/home/xandao/Imagens/pr_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[9865, 9884, 9911, 10134, 10178, 10271, 10380,..."
2,amalago,68,/home/xandao/Imagens/pr_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7707, 7723, 8236, 8767, 9678, 9698, 9888, 993..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,9735,Piper,arboreum,Piper,arboreum,Brasil,Brazil,Antonina,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
1,13064,Piper,corcovadensis,Piper,grazielae,Brasil,Brasil,Diamante do Norte,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
2,9821,Piper,corcovadensis,Piper,grazielae,Brasil,Brazil,Bocaiúva do Sul,Paraná,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/pr_dataset/RGB/specific_e...
color,rgb
image_size,256
minimum_image,5
level_name,specific_epithet_trusted
levels,55
total,1356
average,24.65


color: RGB image_size: 256 minimum_image: 10
total of levels: 36 total of images: 1237


...

,0
dst,/home/xandao/Imagens/pr_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,10
level_name,specific_epithet_trusted
levels,36
total,1237
average,34.36


color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 23 total of images: 1066


,levels,count,dst,paths,seq
0,aduncum,33,/home/xandao/Imagens/pr_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[9674, 9927, 10002, 10051, 10094, 10468, 10676..."
1,amalago,68,/home/xandao/Imagens/pr_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7707, 7723, 8236, 8767, 9678, 9698, 9888, 993..."
2,arboreum,35,/home/xandao/Imagens/pr_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8834, 9710, 9735, 9749, 9765, 9776, 9842, 991..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,9735,Piper,arboreum,Piper,arboreum,Brasil,Brazil,Antonina,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
1,13064,Piper,corcovadensis,Piper,grazielae,Brasil,Brasil,Diamante do Norte,Paraná,[https://storage.googleapis.com/cria-zoomify/h...
2,9821,Piper,corcovadensis,Piper,grazielae,Brasil,Brazil,Bocaiúva do Sul,Paraná,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/pr_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,20
level_name,specific_epithet_trusted
levels,23
total,1066
average,46.35


In [7]:
session.close()
engine.dispose()